In [25]:
# define rooms and items
from colorama import Fore, Back, Style

couch = {
    "name": "couch",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
}

key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

piano = {
    "name": "piano",
    "type": "furniture",
}

game_room = {
    "name": "game room",
    "type": "room",
}

outside = {
  "name": "outside"
}

queen_bed= {
    "name": "queen bed",
    "type": "furniture",
}

door_b = {
    "name": "door b",
    "type": "door",
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

double_bed= {
    "name": "double bed",
    "type": "furniture",
}

dresser= {
    "name": "dresser",
    "type": "furniture",
}

bedroom_1 = {
    "name": "bedroom 1",
    "type": "room",
}

door_c = {
    "name": "door c",
    "type": "door",
}

dining_table= {
    "name": "dining table",
    "type": "furniture",
}

door_d = {
    "name": "door d",
    "type": "door",
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": door_d,
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

bedroom_2 = {
    "name": "bedroom 2",
    "type": "room",
}

living_room = {
    "name": "living room",
    "type": "room",
}

all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, door_a],
    "bedroom 1": [queen_bed, door_b, door_a, door_c],
    "bedroom 2": [double_bed, door_b, dresser],
    "living room": [dining_table, door_c, door_d],
    "piano": [key_a],
    "queen bed": [key_b],
    "double bed": [key_c],
    "dresser": [key_d],
    "outside": [door_d],
    "door a": [game_room, bedroom_1],
    "door b": [bedroom_1, bedroom_2],
    "door c": [bedroom_1, living_room],
    "door d": [living_room, outside],
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside,
}

EMPTY_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside,
}


In [26]:
def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print(Fore.BLACK+"You wake up after"+Fore.BLUE+" an IRONBEERS night " +Fore.BLACK +"on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. "+Fore.BLUE+"Normal... you drunk as if there was no tomorrow..."+Fore.BLACK+" Now you want to get the hell out of there ASAP, so you better start moving!"+Fore.RESET)
    EMPTY_GAME_STATE = INIT_GAME_STATE.copy()
    game_state = EMPTY_GAME_STATE.copy()
    play_room(game_state["current_room"])

def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room! ")
        finish=play_again(object_relations)
        if finish == True:
            #print("Thanks for playing, see you next time")
            crazy_pacman()
        elif finish == False:
            start_game()
    else:
        print("You are now in " + room["name"])
        intended_action = input(Fore.BLACK+"What would you like to do? Type"+Fore.BLUE+ "'explore' "+Fore.BLACK +"or"+Fore.BLUE +" 'examine'? "+Fore.RESET).strip().lower()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine? ").strip().lower())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'. ")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print(Fore.GREEN+"You explore the room. This is " + room["name"] + ". You find " + ", ".join(items)+Fore.RESET)

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(current_room != room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = Fore.BLACK + "You examine " + item_name + ". "+Fore.RESET
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += Fore.GREEN+"You unlock it with a key you have. "+Fore.RESET
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += Fore.RED + "It is locked but you don't have the key. "+Fore.RESET
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += Fore.YELLOW + Back.BLACK+"You find " + item_found["name"] + "."+Fore.RESET+Back.RESET
                else:
                    output += Fore.BLACK+ "There isn't anything interesting about it."+Fore.RESET
            print(output)
            break

    if(output is None):
        print(Fore.BLACK+"The item you requested is not found in the current room."+Fore.RESET)
    ok=0
    if next_room:
        while ok==0:
            inpt=input(Fore.BLUE+"Do you want to go to the next room? Enter 'yes' or 'no'").strip().lower()
            if(inpt == 'yes'):
                ok=1
                play_room(next_room)
            elif(inpt == 'no'):
                ok=1
                play_room(current_room)
            else:
                print(Fore.RED+"Please write yes or no "+Fore.RESET)
    else:
        play_room(current_room)
        
def play_again(dicti):
    linebreak()
    action=input(Fore.WHITE+Back.BLACK+"Do you want to play again? ").lower()
    while True:    
        if action=="yes":
            init_object_rel(dicti)
            return False
        elif action=="no":
            return True
        else:
            print(Fore.RED+"Please write yes or no "+Fore.RESET)
            action=input(Fore.WHITE+Back.BLACK+"Do you want to play again? "+Fore.RESET+Back.RESET).lower()
            
def init_object_rel(dicti):
    newdicti={"piano": [key_a], "queen bed": [key_b], "double bed": [key_c], "dresser": [key_d]}
    dicti.update(newdicti)

def crazy_pacman():
    print('''
                  THANKS    FOR     PLAYING
        ================================================.
             .-.   .-.     .--.                         |
            | OO| | OO|   / _.-' .-.   .-.  .-.   .''.  |
            |   | |   |   \  '-. '-'   '-'  '-'   '..'  |
            '^^^' '^^^'    '--'                         |
        ===============.  .-.  .================.  .-.  |
                       | |   | |                |  '-'  |
                       | |   | |                |       |
                       | ':-:' |                |  .-.  |
                       |  '-'  |                |  '-'  |
        ==============='       '================'       |
    ''')

In [27]:
game_state = EMPTY_GAME_STATE.copy()

start_game()

### CHANGES THAT HAVE BEEN MADE:
    # You can play endelessly if you want
    # You can enter everything in CAPITAL LETERS
    # If you want to open a door and you don't enter yes/YES or no/NO then it will keep asking you
    # Colors have been changed depending on the progress you are making (e.g. find a key, unlocking a door, or error while responding)

You wake up after an IRONBEERS night on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. Normal... you drunk as if there was no tomorrow... Now you want to get the hell out of there ASAP, so you better start moving!
You are now in game room
What would you like to do? Type'explore' or 'examine'? explore
You explore the room. This is game room. You find couch, piano, door a
You are now in game room
What would you like to do? Type'explore' or 'examine'? examine
What would you like to examine? PIANo
You examine piano. You find key for door a.
You are now in game room
What would you like to do? Type'explore' or 'examine'? examine
What would you like to examine? door a
You examine door a. You unlock it with a key you have. 
Do you want to go to the next room? Enter 'yes' or 'no'yes
You are now in bedroom 1
What would you like to do? Type'explore' or 'examine'? explore
You explor